In [18]:
import xarray as xr
import ocf_blosc2
import pandas as pd
import warnings
warnings.filterwarnings("ignore")
# import matplotlib.pyplot as""" plt
# import plotly.express as px
import numpy as np


In [19]:
forecast_data = pd.read_csv("result_data/final_data_tft2.csv")
forecast_data = forecast_data.drop(columns=['Unnamed: 0'])
forecast_data['pv_datetime'] = pd.to_datetime(forecast_data['pv_datetime'])
forecast_data['date'] = forecast_data['pv_datetime'].dt.date
forecast_data['day_of_week'] = forecast_data['pv_datetime'].dt.dayofweek
forecast_data['month'] = forecast_data['pv_datetime'].dt.month


In [25]:
forecast_data

,ss_id,lat,long,tilt,orientation,dlwrf,dswrf,duvrs,hcc,lcc,...,u10,u100,v10,v100,day_of_week,month,generation,pv_datetime,date,pv_hour
0,4127,51.57,-1.83,30.0,25.0,14962224.0,1819869.90,201821.34,0.993835,0.092926,...,2.260745,4.646712,4.038722,6.724235,4,2,429.221985,2020-02-14 12:00:00,2020-02-14,1
1,4127,51.57,-1.83,30.0,25.0,16579950.0,883466.94,126391.64,0.998474,0.107025,...,3.932987,6.590419,11.055776,17.039238,4,2,478.260010,2020-02-14 13:00:00,2020-02-14,2
2,4127,51.57,-1.83,30.0,25.0,17810366.0,1310677.50,176970.19,0.757904,1.000000,...,3.309869,5.515859,11.820305,18.185806,4,2,399.561615,2020-02-14 14:00:00,2020-02-14,3
3,4127,51.57,-1.83,30.0,25.0,19100172.0,1414303.60,194360.25,0.999268,0.998444,...,5.633360,9.146906,12.324112,18.776012,4,2,379.167603,2020-02-14 15:00:00,2020-02-14,4
4,4127,51.57,-1.83,30.0,25.0,20402764.0,1492403.50,204789.03,0.999725,1.000000,...,4.498135,7.335858,11.324584,17.546019,4,2,54.918961,2020-02-14 16:00:00,2020-02-14,5
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
10191,27057,51.52,-0.43,40.0,225.0,20342560.0,8549557.00,857421.00,0.999969,0.001434,...,-0.939994,-1.015804,4.964947,9.304050,2,10,0.000000,2021-10-27 19:00:00,2021-10-27,1
10192,27057,51.52,-0.43,40.0,225.0,21435648.0,8549557.00,857421.00,0.999939,0.082306,...,-0.615427,-0.506306,5.241445,9.604069,2,10,0.000000,2021-10-27 20:00:00,2021-10-27,2
10193,27057,51.52,-0.43,40.0,225.0,22558016.0,8550104.00,857421.00,1.000000,0.073273,...,-0.440664,-0.158624,6.032712,10.556226,2,10,0.000000,2021-10-27 21:00:00,2021-10-27,3
10194,27057,51.52,-0.43,40.0,225.0,23721456.0,8550104.00,857373.70,0.996948,0.584595,...,0.012856,0.566574,6.684642,11.339936,2,10,0.000000,2021-10-27 22:00:00,2021-10-27,4


In [20]:
forecast_data.head()

,ss_id,init_time,step,generation,pv_datetime,pv_hour,lat,long,tilt,orientation,...,sr,t2m,tcc,u10,u100,v10,v100,date,day_of_week,month
0,26906,2021-07-29 12:00:00,0 days 02:00:00,0.000000,2021-07-29 02:00:00,2,53.52,-2.93,26.0,225.0,...,319360.00,289.20044,0.999971,9.506679,12.843964,1.211929,1.569199,2021-07-29,3,7
1,26906,2021-07-29 12:00:00,0 days 03:00:00,0.000000,2021-07-29 03:00:00,3,53.52,-2.93,26.0,225.0,...,451328.00,289.29540,0.998687,8.772396,11.869049,0.832458,1.075440,2021-07-29,3,7
2,26906,2021-07-29 12:00:00,0 days 04:00:00,0.000000,2021-07-29 04:00:00,4,53.52,-2.93,26.0,225.0,...,731648.00,289.56616,0.999325,7.909646,10.661621,0.461304,0.630722,2021-07-29,3,7
3,26906,2021-07-29 12:00:00,0 days 05:00:00,6.474624,2021-07-29 05:00:00,5,53.52,-2.93,26.0,225.0,...,991278.44,289.59302,0.987131,6.859680,9.287048,-0.223587,-0.262619,2021-07-29,3,7
4,26906,2021-07-29 12:00:00,0 days 06:00:00,43.018799,2021-07-29 06:00:00,6,53.52,-2.93,26.0,225.0,...,1281776.10,289.50270,0.746552,6.261519,8.631668,-0.405579,-0.522446,2021-07-29,3,7


In [21]:
forecast_data.columns

Index(['ss_id', 'init_time', 'step', 'generation', 'pv_datetime', 'pv_hour',
       'lat', 'long', 'tilt', 'orientation', 'dlwrf', 'dswrf', 'duvrs', 'hcc',
       'lcc', 'mcc', 'sde', 'sr', 't2m', 'tcc', 'u10', 'u100', 'v10', 'v100',
       'date', 'day_of_week', 'month'],
      dtype='object')

In [22]:
target_variable = 'generation'
static_features = ['ss_id', 'lat', 'long', 'tilt', 'orientation']
known_future_inputs = ['dlwrf', 'dswrf', 'duvrs', 'hcc', 'lcc', 'mcc', 'sde', 'sr', 't2m', 'tcc', 'u10', 'u100', 'v10', 'v100', 'day_of_week', 'month']
required_columns = static_features + known_future_inputs + [target_variable, 'pv_datetime', 'date']

forecast_data = forecast_data[required_columns]

forecast_data = forecast_data.fillna(method='bfill').fillna(method='ffill')



In [23]:
def generate_pv_hours(group):
    group = group.sort_values(by='pv_datetime').reset_index(drop=True)
    group['pv_hour'] = group.groupby('date').cumcount() + 1
    return group

# Apply the function to each group
forecast_data = forecast_data.groupby('ss_id').apply(generate_pv_hours).reset_index(drop=True)


In [26]:
forecast_data.to_csv("test2.csv")